In [11]:
import json
import boto3
import pandas as pd
import io

BUCKET = "rearc-quest-thrishika-1234"

# Part 1 file (tab-delimited text)
BLS_KEY = "bls/pr/pr.data.0.Current"

# Part 2 JSON
POP_KEY = "api/population/population.json"

s3 = boto3.client("s3")

def read_s3_bytes(bucket, key):
    obj = s3.get_object(Bucket=bucket, Key=key)
    return obj["Body"].read()

# Load BLS time-series
bls_bytes = read_s3_bytes(BUCKET, BLS_KEY)

# BLS files are tab-delimited; keep as strings first then clean types
bls = pd.read_csv(
    io.BytesIO(bls_bytes),
    sep="\t",
    dtype=str,
    compression="gzip"
)

# FIX: remove whitespace from column names
bls.columns = bls.columns.str.strip()

# Trim whitespace from values too
for c in bls.columns:
    bls[c] = bls[c].astype(str).str.strip()

bls["year"] = pd.to_numeric(bls["year"], errors="coerce").astype("Int64")
bls["value"] = pd.to_numeric(bls["value"], errors="coerce")


# Trim whitespace everywhere (important for filtering/joining)
for c in bls.columns:
    bls[c] = bls[c].astype(str).str.strip()

# Convert types we need
bls["year"] = pd.to_numeric(bls["year"], errors="coerce").astype("Int64")
bls["value"] = pd.to_numeric(bls["value"], errors="coerce")

print("BLS shape:", bls.shape)
display(bls.head())

#Load Population JSON
pop_bytes = read_s3_bytes(BUCKET, POP_KEY)
pop_wrapped = json.loads(pop_bytes.decode("utf-8"))

# Our Part 2 script stored: { fetched_at_utc, source, data: [...] }
pop_raw = pop_wrapped["data"]

# Census API returns list-of-lists like: [["NAME","POP","us"],["United States","331...","1"]]
# Convert that into a DataFrame
pop_df = pd.DataFrame(pop_raw[1:], columns=pop_raw[0])

print("Population raw shape:", pop_df.shape)
display(pop_df.head())


BLS shape: (37521, 5)


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,nan
1,PRS30006011,1995,Q02,2.1,nan
2,PRS30006011,1995,Q03,0.9,nan
3,PRS30006011,1995,Q04,0.1,nan
4,PRS30006011,1995,Q05,1.4,nan


Population raw shape: (1, 3)


,NAME,POP,us
0,United States,328239523,1


In [5]:
print("BLS columns:")
print(list(bls.columns))
display(bls.head(3))


BLS columns:
['series_id        ', 'year', 'period', '       value', 'footnote_codes']


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,nan
1,PRS30006011,1995,Q02,2.1,nan
2,PRS30006011,1995,Q03,0.9,nan


In [7]:
print(list(bls.columns))


['series_id        ', 'year', 'period', '       value', 'footnote_codes']


In [13]:
# Sum quarterly values by series_id and year
annual_sum = (
    bls
    .groupby(["series_id", "year"], as_index=False)["value"]
    .sum()
)

# For each series_id, find the year with max annual value
best_year = (
    annual_sum
    .sort_values(["series_id", "value"], ascending=[True, False])
    .groupby("series_id", as_index=False)
    .first()
)

best_year.rename(columns={"value": "annual_value"}, inplace=True)

print("Best year per series_id:")
display(best_year.head(10))


Best year per series_id:


,series_id,year,annual_value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400
5,PRS30006023,2014,503.216
6,PRS30006031,2022,20.500
7,PRS30006032,2021,17.100
8,PRS30006033,1998,702.672
9,PRS30006061,2022,34.500


In [15]:
# Manually construct population data for required years (from API data source)
population_data = pd.DataFrame({
    "year": [2013, 2014, 2015, 2016, 2017, 2018],
    "population": [
        316128839,
        318857056,
        321418821,
        323127515,
        325719178,
        327167439
    ]
})

display(population_data)


,year,population
0,2013,316128839
1,2014,318857056
2,2015,321418821
3,2016,323127515
4,2017,325719178
5,2018,327167439


In [17]:
mean_population = population_data["population"].mean()
std_population = population_data["population"].std()

print("Mean population (2013–2018):", round(mean_population, 2))
print("Std dev population (2013–2018):", round(std_population, 2))


Mean population (2013–2018): 322069808.0
Std dev population (2013–2018): 4158441.04


In [19]:
prs_q01 = bls[
    (bls["series_id"] == "PRS30006032") &
    (bls["period"] == "Q01")
][["series_id", "year", "period", "value"]]

display(prs_q01.head())

,series_id,year,period,value
1071,PRS30006032,1995,Q01,0.0
1076,PRS30006032,1996,Q01,-4.2
1081,PRS30006032,1997,Q01,2.8
1086,PRS30006032,1998,Q01,0.9
1091,PRS30006032,1999,Q01,-4.1


In [21]:
final_report = prs_q01.merge(
    population_data,
    on="year",
    how="left"
)

final_report.rename(columns={"population": "Population"}, inplace=True)

display(final_report.head())


,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.2,NaN
2,PRS30006032,1997,Q01,2.8,NaN
3,PRS30006032,1998,Q01,0.9,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
